In [1]:
import xarray as xr               #netcdf multidim reading/writing/manipulation
import glob                       #globbing
import numpy as np                #numerics
import os                         #operating system commands
import subprocess
import time as timer
import pop_tools
import sys

In [2]:
indir = '/glade/collections/cdg/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-piControl.001/ocn/proc/tseries/month_1'
outdir = '/glade/scratch/yeager'
case = 'b.e21.B1850.f09_g17.CMIP6-piControl.001'
tstmp = '000101-009912'
fout = f'{outdir}/{case}.pop.h.SDEN_F.{tstmp}.nc'

ht_vars = ['SHF', 'SHF_QSW', 'LWDN_F', 'LWUP_F', 'SENH_F', 'MELTH_F', 'QFLUX']
fw_vars = ['SFWF', 'EVAP_F', 'PREC_F', 'ROFF_F', 'IOFF_F', 'MELT_F', 'SALT_F', 'SNOW_F']
fin_ht = [f'{indir}/{case}.pop.h.' + var + f'.{tstmp}.nc' for var in ht_vars]
fin_fw = [f'{indir}/{case}.pop.h.' + var + f'.{tstmp}.nc' for var in fw_vars]
fin_ts = [f'{indir}/{case}.pop.h.TEMP.{tstmp}.nc',f'{indir}/{case}.pop.h.SALT.{tstmp}.nc']

In [3]:
#Load flux datasets
ds_ht = xr.open_mfdataset(fin_ht, combine='by_coords')
ds_fw = xr.open_mfdataset(fin_fw, combine='by_coords')

In [4]:
ds_ht

<xarray.Dataset>
Dimensions:                 (d2: 2, lat_aux_grid: 395, moc_comp: 3, moc_z: 61, nlat: 384, nlon: 320, time: 1188, transport_comp: 5, transport_reg: 2, z_t: 60, z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
Coordinates:
  * z_t                     (z_t) float32 500.0 1500.0 ... 512502.8 537500.0
  * z_t_150m                (z_t_150m) float32 500.0 1500.0 ... 13500.0 14500.0
  * z_w                     (z_w) float32 0.0 1000.0 ... 500004.7 525000.94
  * z_w_top                 (z_w_top) float32 0.0 1000.0 ... 500004.7 525000.94
  * z_w_bot                 (z_w_bot) float32 1000.0 2000.0 ... 549999.06
  * lat_aux_grid            (lat_aux_grid) float32 -79.48815 -78.952896 ... 90.0
  * moc_z                   (moc_z) float32 0.0 1000.0 ... 525000.94 549999.06
    ULONG                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ULAT                    (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    TLONG        

In [5]:
#conversion factors
latvap=ds_fw['latent_heat_vapor']       # J/kg
latfus=ds_fw['latent_heat_fusion']      # erg/g
latfus = latfus/1.e4
latfus.attrs['units'] = 'J/kg'
latfus

<xarray.DataArray 'latent_heat_fusion' ()>
array(333700.)
Attributes:
    units:    J/kg

In [6]:
#Modify heat flux dataset
ds_ht['SHF'] = ds_ht['SHF'] + ds_ht['QFLUX']
ds_ht['LWDN_F'] = ds_ht['LWDN_F'] + ds_ht['LWUP_F']
ds_ht=ds_ht.rename({'LWDN_F':'LW'})
ds_ht=ds_ht.drop('LWUP_F')

In [7]:
latent=ds_fw['EVAP_F']*latvap
latent.attrs = {'units':'W/m^2','long_name':'Latent Heat Flux'}
ds_ht['Latent']=latent
snowmelt=-ds_fw['SNOW_F']*latfus
snowmelt.attrs = {'units':'W/m^2','long_name':'SNOW_F Heat Flux'}
ds_ht['Snowmelt']=snowmelt
ioffmelt=-ds_fw['IOFF_F']*latfus
ioffmelt.attrs = {'units':'W/m^2','long_name':'IOFF_F Heat Flux'}
ds_ht['Ioffmelt']=ioffmelt

In [8]:
ds_ht

<xarray.Dataset>
Dimensions:                 (d2: 2, lat_aux_grid: 395, moc_comp: 3, moc_z: 61, nlat: 384, nlon: 320, time: 1188, transport_comp: 5, transport_reg: 2, z_t: 60, z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
Coordinates:
  * z_t                     (z_t) float32 500.0 1500.0 ... 512502.8 537500.0
  * z_t_150m                (z_t_150m) float32 500.0 1500.0 ... 13500.0 14500.0
  * z_w                     (z_w) float32 0.0 1000.0 ... 500004.7 525000.94
  * z_w_top                 (z_w_top) float32 0.0 1000.0 ... 500004.7 525000.94
  * z_w_bot                 (z_w_bot) float32 1000.0 2000.0 ... 549999.06
  * lat_aux_grid            (lat_aux_grid) float32 -79.48815 -78.952896 ... 90.0
  * moc_z                   (moc_z) float32 0.0 1000.0 ... 525000.94 549999.06
    ULONG                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ULAT                    (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    TLONG        

In [7]:
#Modify freshwater flux dataset
ds_fw['SFWF'] = ds_ht['SFWF'] - ds_ht['QFLUX']/latfus

KeyError: 'SFWF'

In [9]:
ds_ts = xr.open_mfdataset(fin_ts, combine='by_coords').isel(z_t=0)
#ds_ts

In [14]:
# Compute alpha & beta
tmps = ds_ts['SALT'].isel(time=0)
tmpt = ds_ts['TEMP'].isel(time=0)
depth=xr.DataArray(np.zeros(np.shape(tmpt)),dims=tmpt.dims,coords=tmpt.coords)
#rho,drhods,drhodt = pop_tools.eos(salt=tmps,temp=tmpt,return_coefs=True,depth=depth)
#depth=0.
rho = pop_tools.eos(salt=tmps,temp=tmpt,depth=depth)

In [10]:
rho

<xarray.DataArray 'density' (nlat: 384, nlon: 320)>
dask.array<compute_eos, shape=(384, 320), dtype=float32, chunksize=(384, 320)>
Coordinates:
    z_t      float32 500.0
    ULONG    (nlat, nlon) float64 321.1 322.3 323.4 324.5 ... 319.2 319.6 320.0
    ULAT     (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.42 72.41 72.41
    TLONG    (nlat, nlon) float64 320.6 321.7 322.8 323.9 ... 318.9 319.4 319.8
    TLAT     (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.2 72.19 72.19
    time     object 0001-02-01 00:00:00
Dimensions without coordinates: nlat, nlon
Attributes:
    units:      kg/m^3
    long_name:  Density